# Neural Network from Scratch in TensorFlow

# Task 1: Introduction and Importing Libraries

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nn_utils
%matplotlib inline

print('TensorFlow Version:', tf.__version__)

TensorFlow Version: 2.12.0


# Task 2: Initializing Neural Network

In [4]:
class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.L = len(layers)
        self.num_features = layers[0] ##Number of features
        self.num_classes = layers[-1] ##Number of otput classes
        
        self.W = {}
        self.b = {}
        
        #db is bias
        self.dW = {}
        self.db = {}
        
        self.setup()
        
    def setup(self):
        for i in range(1,self.L):
          self.W[i] = tf.Variable(tf.random.normal(shape=(self.layers[i],self.layers[i-1])))
          self.b[i]=tf.Variable(tf.random.normal(shape=(self.layers[i],1)))

# Task 3: Forward Pass

In [6]:
class NeuralNetwork(NeuralNetwork):
    def forward_pass(self, X): # X is input here
        A = tf.convert_to_tensor(X,dtype=tf.float32)
        for i in range(1,self.L):
          Z = tf.matmul(A,tf.transpose(self.W[i]))+tf.transpose(self.b[i])# Z is the linear output
          if i != self.L-1: # If the layers is not the final layer
            # Setting up activation for hidden layers
            A = tf.nn.relu(Z)
          else: 
            # Setting up AF for final layer
            A = Z
        return A

# Task 4: Computing Loss and Updating Parameters

In [7]:
class NeuralNetwork(NeuralNetwork):
    def compute_loss(self,A,Y): # Y is ground truth , while A is the final output
      loss = tf.nn.softmax_cross_entropy_with_logits(Y,A)
      return tf.reduce_mean(loss)

    def update_params(self,lr):
      for i in range(1,self.L):
        self.W[i].assign_sub(lr * self.dW[i])
        self.b[i].assign_sub(lr * self.db[i])

# Task 5: Predict and Info Functions

In [8]:
class NeuralNetwork(NeuralNetwork):
    def predict(self, X):
        A = self.forward_pass(X) # getting final output
        return tf.argmax(tf.nn.softmax(A),axis=1)  # applying softmax
    
    def info(self):
        num_params = 0
        for i in range(1, self.L):
            num_params += self.W[i].shape[0] * self.W[i].shape[1]
            num_params += self.b[i].shape[0]
        print('Input Features:', self.num_features)
        print('Number of Classes:', self.num_classes)
        print('Hidden Layers:')
        print('--------------')
        for i in range(1, self.L-1):
            print('Layer {}, Units {}'.format(i, self.layers[i]))
        print('--------------')
        print('Number of parameters:', num_params)

# Task 6: Training on Batch

In [9]:
from tensorflow.python.eager.backprop import GradientTape
class NeuralNetwork(NeuralNetwork):
    def train_on_batch(self, X, Y, lr):
        # X and Y might be in numpy so we have to covert these into tensors
        X = tf.convert_to_tensor(X,dtype=tf.float32)
        Y = tf.convert_to_tensor(Y,dtype=tf.float32)

        with tf.GradientTape(persistent=True) as tape:
          A = self.forward_pass(X
                                )
          loss=self.compute_loss(A,Y)
        
        for i in range(1,self.L):
          self.dW[i]=tape.gradient(loss,self.W[i])
          self.db[i]=tape.gradient(loss,self.b[i])

        del tape
        self.update_params(lr)
        return loss.numpy() 

# Task 7: Training on Complete Set

In [10]:
class NeuralNetwork(NeuralNetwork):
    def train(self, x_train, y_train, x_test, y_test, epochs, steps_per_epoch, batch_size, lr):
        # 
        history = {
            'val_loss':[],
            'train_loss':[],
            "val_acc":[]
        }

        # Breaking into mini batches
        for e in range(0 , epochs):
          epoch_train_loss=0
          print("Epoch {}".format(e),end='.')
          for i in range(0,steps_per_epoch):
            x_batch=x_train[i*batch_size:(i+1)*batch_size]
            y_batch=y_train[i*batch_size:(i+1)*batch_size]

            batch_loss = self.train_on_batch(x_batch,y_batch,lr)
            epoch_train_loss+=batch_loss

            if i%int(steps_per_epoch/10)  == 0:
              print(end='.')

          history['train_loss'].append(epoch_train_loss/steps_per_epoch)
            
          val_A=self.forward_pass(x_test)
          val_loss=self.compute_loss(val_A,y_test).numpy()
          history['val_loss'].append(val_loss)
            
          val_pred = self.predict(x_test)
          val_acc=np.mean(np.argmax(y_test,axis=1) == val_pred.numpy())
          history['val_acc'].append(val_acc)

        print("Val Acc:",val_acc)



# Task 8: Application

# Task 9: Results